# Overview

In this notebook, you leverage the cpdctl command to copy assets from one deployment space of Cloud Pak for Data to another deployment space. This is typically applicable in MLOps process where you want to promote the relevant data assets and AI models from an initial deployment space, such as Pre-Prod (or UAT) deployment space, to another deployment space, such as Prod deployment space.

For purposes of AI governance as well as CI/CD component of MLOps, it is important to be able to automate the process of promoting relevant assets from one deployment space owner by development team to another deployment space owned by the QA team.

Please note that the two deployment spaces could exist on the same Cloud Pak for Data cluster or could belong to two different Cloud Pak for Data clusters in different environments.
Execution Steps

In order to leverage cpdctl to copy assets from one deployment space to another, you need to provide the following information:

- SOURCE_CPD_URL : The url for the source Cloud Pak for Data cluster
- SOURCE_CPD_USERNAME: The username for the source Cloud Pak for Data cluster
- SOURCE_CPD_PASSWORD: The password for the source Cloud Pak for Data cluster
- TARGET_CPD_URL : The url for the target Cloud Pak for Data cluster
- TARGET_CPD_USERNAME: The username for the target Cloud Pak for Data cluster
- TARGET_CPD_PASSWORD: The password for the target Cloud Pak for Data cluster
- SOURCE_DEPLOYMENT_SPACE_NAME: The name of the deployment space on the source Cloud Pak for Data cluster (source deployment space should exist).
- TARGET_DEPLOYMENT_SPACE_NAME: The name of the deployment space on the target Cloud Pak for Data cluster (any name you choose since it will be created).
- TARGET_MODEL_NAME: The name of the model in the atarget deployment space (any name you choose since it will be created).

Given this information, this notebook will define the cpdctl contexts corresponding to the source and target Cloud Pak for Data clusters, create a new deployment space on the target cluster, and copy all assets from the source deployment space to the target deployment space.

The code assumes that the SOURCE_DEPLOYMENT_SPACE_NAME exists and contains the relevant assets and that the TARGET_DEPLOYMENT_SPACE_NAME also exists but has no assets.

Also please NOTE that when running this notebook inside IBM Cloud Pak for Data (CP4D) cluster, cpdctl takes advantage of [zero-configuration mode](https://github.com/IBM/cpdctl#zero-configuration)  which means it can connect to the CP4D without explicit configuration. In that case, no need to explicitly configure context. Since this notebook is designed to explicitly run within the Cloud Pak for Data cluster, we will comment out the cells for configuring context. If you plan to use cpdctl in automation pipelines external to the cluster, then you need to include these configuration steps.

## Before you begin
Import the following libraries:

In [ ]:
import base64
import json
import os
import requests
import platform
import tarfile
import zipfile
import subprocess
from IPython.core.display import display, HTML

Download the cpdctl binary and then display the version number:

In [ ]:
PLATFORM = platform.system().lower()
CPDCTL_ARCH = "{}_amd64".format(PLATFORM)
CPDCTL_RELEASES_URL="https://api.github.com/repos/IBM/cpdctl/releases"
CWD = os.getcwd()
PATH = os.environ['PATH']
CPD_CONFIG = os.path.join(CWD, '.cpdctl.config.yml')

response = requests.get(CPDCTL_RELEASES_URL)
assets = response.json()[0]['assets']
platform_asset = next(a for a in assets if CPDCTL_ARCH in a['name'])
cpdctl_url = platform_asset['url']
cpdctl_file_name = platform_asset['name']
        
response = requests.get(cpdctl_url, headers={'Accept': 'application/octet-stream'})
with open(cpdctl_file_name, 'wb') as f:
    f.write(response.content)
    
display(HTML('<code>cpdctl</code> binary downloaded from: <a href="{}">{}</a>'.format(platform_asset['browser_download_url'], platform_asset['name'])))

In [ ]:
%%capture

%env PATH={CWD}:{PATH}
%env CPD_CONFIG={CPD_CONFIG}

In [ ]:
if cpdctl_file_name.endswith('tar.gz'):
    with tarfile.open(cpdctl_file_name, "r:gz") as tar:
        tar.extractall()
elif cpdctl_file_name.endswith('zip'):
    with zipfile.ZipFile(cpdctl_file_name, 'r') as zf:
        zf.extractall()

if CPD_CONFIG and os.path.exists(CPD_CONFIG):
    os.remove(CPD_CONFIG)
    
version_r = ! cpdctl version
CPDCTL_VERSION = version_r.s

print("cpdctl version: {}".format(CPDCTL_VERSION))

In [ ]:
# Code assumes both deployment spaces exist
SOURCE_DEPLOYMENT_SPACE_NAME='churnUATspace' # For example: 'churnUATspace'
TARGET_DEPLOYMENT_SPACE_NAME='churn_prod_space' # For example: 'churn_prod_space'

# Provide the ML Model name in the source deployment space (model must exist)
# and the desired Deployment Model name in the target deployment space
SOURCE_MODEL_NAME='Churn Model'
TARGET_DEPLOYMENT_NAME='Churn_Deployment_Prod'

List available spaces in context:

In [ ]:
! cpdctl space list

In [ ]:
def getSpaceID(name):
    cmd="cpdctl space list --output json" + " --jmes-query \"resources[?entity.name == " + "'" + name + "'" + "].metadata.id\""  
    print("executing command: ", cmd)
    
    result = subprocess.getoutput(cmd)
    space_id=json.loads(result)
    if len(space_id) != 1:
        print("Error, found ", len(space_id), " spaces with the name: ", name)
    return space_id[0]

In [ ]:
source_deployment_spaceID=getSpaceID(SOURCE_DEPLOYMENT_SPACE_NAME)
print("Source Deployment Space ID: ", source_deployment_spaceID)

In [ ]:
source_ml_models = ! cpdctl ml model list --space-id {source_deployment_spaceID} --output json --jmes-query "resources[*].metadata"
ML_MODEL_IDS = json.loads(source_ml_models.s)
for mlmodel in ML_MODEL_IDS:
    if mlmodel['name'] == SOURCE_MODEL_NAME:
        MODEL_ID = mlmodel['id']

In [ ]:
source_ml_models

In [ ]:
MODEL_ID

In [ ]:
# Export the model from source deployment space
EXPORT = {
    'asset_ids': [MODEL_ID]
}
EXPORT_JSON = json.dumps(EXPORT)
##! cpdctl config context use source
result = ! cpdctl asset export start --space-id {source_deployment_spaceID} --assets '{EXPORT_JSON}' --name source-model --output json --jmes-query "metadata.id"
EXPORT_ID = result.s
print("The new export with ID: {}".format(EXPORT_ID))

In [ ]:
# Download exported source model as zip file
! cpdctl asset export download --space-id {source_deployment_spaceID} --export-id {EXPORT_ID} --output-file source-model.zip

In [ ]:
TARGET_SPACE_ID=getSpaceID(TARGET_DEPLOYMENT_SPACE_NAME)
print("Target Deployment Space ID: ", TARGET_SPACE_ID)

In [ ]:
result = ! cpdctl asset import start --space-id {TARGET_SPACE_ID} --import-file source-model.zip --output json --jmes-query "metadata.id" --raw-output
IMPORT_ID = result.s
print("The new import ID is: {}".format(IMPORT_ID))


In [ ]:
! cpdctl asset import get --space-id {TARGET_SPACE_ID} --import-id {IMPORT_ID}

In [ ]:
! cpdctl ml model list --space-id {TARGET_SPACE_ID}

In [ ]:
result = ! cpdctl ml model list --space-id {TARGET_SPACE_ID} --output json --jmes-query "resources[0].metadata.id" --raw-output
TARGET_MODEL_ID = result.s
print("TARGET model ID is: {}".format(TARGET_MODEL_ID))

In [ ]:
ASSET_JSON = json.dumps({"id": TARGET_MODEL_ID})
ONLINE_JSON = json.dumps({})

! cpdctl ml deployment create --space-id {TARGET_SPACE_ID} --asset '{ASSET_JSON}' --online '{ONLINE_JSON}' --name {TARGET_DEPLOYMENT_NAME}


## Verify Deployment

At this point, a new model deployment should appear in your target deployment space. The following steps explain how to verify and test the model using the UI.

- Navigate to your target deployment space: Select the Navigation Menu (top left hamburger icon), right click on Deployments, and select Open Link in New Tab.
- In the new tab, select the Spaces tab and click on the name of the target deployment space (for example, churn_prod_space)
- On the Deployments/\<target space name> page, click on Deployments tab.
- Verify the TARGET_MODEL_NAME appears in the list of deployed models. Click the TARGET_MODEL_NAME model.
- On the Deployed model page, click the Test tab and provide a sample test to validate the model returns predictions as expected.

In the remaining cells, we run model deployment verification using Python client.



In [ ]:
cpdtoken=os.environ['USER_ACCESS_TOKEN']
wml_credentials = {
"token": cpdtoken,
"instance_id" : "openshift",
"url": os.environ['RUNTIME_ENV_APSX_URL'],
"version": "4.0"
}

from ibm_watson_machine_learning import APIClient
client = APIClient(wml_credentials)

In [ ]:
def getSpaceIDwml(wml_client,space_name):
    spaces = wml_client.spaces.get_details()['resources'];
    spaceList = next(item for item in spaces if item['entity']['name']==space_name)
    spaceID = spaceList['metadata']['id']
    return spaceID

In [ ]:
space_name=TARGET_DEPLOYMENT_SPACE_NAME
space_id = getSpaceIDwml(client,space_name)
print(space_id)
client.set.default_space(space_id)

In [ ]:
space_details=client.spaces.get_details(space_id)

In [ ]:
client.repository.list_models()

In [ ]:
def getModelDetails(wml_client,deployment_name):
    models = wml_client.deployments.get_details()['resources'];
    modelList = next(item for item in models if item['entity']['name']==deployment_name)
    #modelID = modelList['metadata']['id']
    #return modelID
    return modelList
    

In [ ]:
model_name=TARGET_DEPLOYMENT_NAME
model_details = getModelDetails(client,model_name)
print(model_details)

In [ ]:
# Score the model on a test dataset
scoring_payload = {
    "input_data": [{
        'fields': ['ID', 'LONGDISTANCE', 'INTERNATIONAL', 'LOCAL', 'DROPPED', 'PAYMETHOD', 'LOCALBILLTYPE', 'LONGDISTANCEBILLTYPE', 'USAGE', 'RATEPLAN', 'GENDER','STATUS', 'CHILDREN', 'ESTINCOME', 'CAROWNER', 'AGE'],
        'values': [[1,28,0,60,0,"Auto","FreeLocal","Standard",89,4,"F","M",1,23000,"N",45]]}]
}

In [ ]:
modelID=model_details['metadata']['id']
predictions = client.deployments.score(modelID, scoring_payload)
print(json.dumps(predictions, indent=2))


## Summary

This notebook illustrates one approach to apply CI/CD against your models where you can automate continuous integration and delivery of models from UAT (or preProd) deployment space to production deployment space.


Copyright © 2022 IBM. This notebook and its source code are released under the terms of the MIT License.